# Preprocessing for Classification


In [41]:
# setup - load packages
import pandas as pd
from datasets import Dataset

## A) speeches without party self-mentions

In [42]:
# read data
preprocessed_data = pd.read_csv("../data/final_data.csv")
# Selecting relevant df columns
preprocessed_data = preprocessed_data[['speech_text', 'party']]
# renaming label category
preprocessed_data = preprocessed_data.rename(columns={'party': 'label'})

#### find mentions of own party in the speeches

In [43]:
import re
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt', quiet=True)
nltk.download('punkt_tab',quiet=True)

def extract_party_mentions(speech_text, party):
    
    if not isinstance(speech_text, str):
        return print("nope")

    # make sentences
    sentences = sent_tokenize(speech_text, language='german')
    
    # party to lower
    party_lower = party.lower()
    
    # filter sent where own party is mentioned
    matched = [s for s in sentences if party_lower in s.lower()]
    
    return matched


##### Test on one speech

In [44]:
extract_party_mentions(preprocessed_data['speech_text'].loc[0],preprocessed_data['label'].loc[0])

['Deshalb darf die europäische Bankenunion kein Transferinstrument werden, und, meine Damen und Herren, die CDU/CSU plant dies auch nicht.']

##### Get matches on all speeches as df

In [45]:
results = []

for idx, speech in preprocessed_data.iterrows():
    party = speech['label']
    matches = extract_party_mentions(speech['speech_text'], party)
    if matches:  
        for match in matches:
            results.append({'party': party, 'match': match})

df_matches = pd.DataFrame(results)

In [46]:
df_matches

,party,match
0,CDU/CSU,Deshalb darf die europäische Bankenunion kein ...
1,GRÜNE,"Was wir Grünen vorschlagen, ist, dass wir jetz..."
2,FDP,"Sie müssen der FDP nicht glauben, dass das ein..."
3,LINKE,Man traut sich das als Linker kaum zu sagen: D...
4,LINKE,Eine Bankenunion als Vollkaskoversicherung für...
...,...,...
18347,GRÜNE,"Es ist vollkommen klar, dass wir auch bei der ..."
18348,SPD,Deswegen würde es die SPD-Fraktion auch sehr b...
18349,GRÜNE,Deswegen haben wir uns als Ampel ja auch vorge...
18350,LINKE,Das Wort hat die Kollegin Clara Bünger für die...


In [47]:
df_matches['party'].value_counts()

party
AfD        4668
SPD        3795
LINKE      3768
GRÜNE      2425
CDU/CSU    2126
FDP        1570
Name: count, dtype: int64

#### remove all own party mentions

In [48]:
def extract_and_remove_party_mentions(speech_text, party):
    if not isinstance(speech_text, str):
        return [], speech_text

    sentences = sent_tokenize(speech_text, language='german')
    party_lower = party.lower()

    matched = [s for s in sentences if party_lower in s.lower()]
    cleaned = [s for s in sentences if party_lower not in s.lower()]

    cleaned_text = ' '.join(cleaned)
    return matched, cleaned_text

In [49]:
results = []
clean_texts = []

for idx, row in preprocessed_data.iterrows():
    matches, cleaned_text = extract_and_remove_party_mentions(row['speech_text'], row['label'])
    results.append({'party': row['label'], 'matches': matches})
    clean_texts.append(cleaned_text)

df_matches = pd.DataFrame(results)
preprocessed_data['speech_text'] = clean_texts


In [50]:
preprocessed_data

,speech_text,label
0,Herr Präsident! Kolleginnen und Kollegen! Die ...,CDU/CSU
1,Sehr geehrter Herr Präsident! Liebe Kolleginne...,SPD
2,Herr Präsident! Liebe Kolleginnen und Kollegen...,CDU/CSU
3,Herr Präsident! Liebe Kolleginnen und Kollegen...,GRÜNE
4,Liebe Kolleginnen und Kollegen! Es ist vorhin ...,FDP
...,...,...
36112,Sehr geehrter Herr Präsident! Liebe Kolleginne...,SPD
36113,Frau Präsidentin! Meine Damen und Herren! Sie ...,GRÜNE
36114,Sehr geehrter Herr Präsident! Meine Damen und ...,AfD
36115,Sehr geehrte Präsidentin! Liebe Kolleginnen un...,LINKE


##### sanity check with find function

In [51]:
results_sanity = []

for idx, speech in preprocessed_data.iterrows():
    party = speech['label']
    matches = extract_party_mentions(speech['speech_text'], party)
    if matches:  # nur, wenn Treffer vorliegen
        for match in matches:
            results_sanity.append({'party': party, 'match': match})

df_matches_sanity = pd.DataFrame(results_sanity)

In [52]:
df_matches_sanity

""


In [53]:
preprocessed_data.to_csv("../data/classifier_data_A.csv",index=False)

## B) clean speeches to only include meaningful parts of speeches

In [28]:
# preparing stopwords
# importing german stopword list from github
# link: https://github.com/solariz/german_stopwords
with open("german_stopwords_full.txt", "r", encoding="utf-8") as g:
    german_stopwords_full = [line.strip() for line in g if not line.lstrip().startswith(";")]
# list of words to add to stopwords:
more_stopwords = ["damen", "herren", "herr", "kollegen", "kolleginnen", "präsident", "präsidentin", "kollege", "kollegin", "verehrte", "verehrten"]
# add list of additional words
german_stopwords_full.extend(more_stopwords)

# convert speeches to new coloumn, small letters and tokenized 
preprocessed_data["tokenized_speeches"] = preprocessed_data["speech_text"].str.lower().str.split()

# get rid of stopwords and special characters
preprocessed_data['tokenized_speeches'] = preprocessed_data['tokenized_speeches'].apply(lambda tokens: [word for word in tokens if word not in german_stopwords_full and word.isalpha()])

# takes at least 4 min

In [29]:
def remove_liebe(tokens):
    while tokens and tokens[0].lower() in ['liebe', 'lieber']:
        tokens = tokens[1:]
    return tokens

In [30]:
preprocessed_data['tokenized_woliebe'] = preprocessed_data['tokenized_speeches'].apply(remove_liebe)

In [31]:
preprocessed_data

,speech_text,label,tokenized_speeches,tokenized_woliebe
0,Herr Präsident! Kolleginnen und Kollegen! Die ...,CDU/CSU,"[bankenunion, stabilität, stabilität, erfahrun...","[bankenunion, stabilität, stabilität, erfahrun..."
1,Sehr geehrter Herr Präsident! Liebe Kolleginne...,SPD,"[liebe, liebe, liebe, gäste, schön, vorsitzend...","[gäste, schön, vorsitzenden, finanzausschusses..."
2,Herr Präsident! Liebe Kolleginnen und Kollegen...,CDU/CSU,"[liebe, thema, bankenunion, verschlossenen, tü...","[thema, bankenunion, verschlossenen, türen, di..."
3,Herr Präsident! Liebe Kolleginnen und Kollegen...,GRÜNE,"[liebe, große, fraktion, geschämt, dargeboten,...","[große, fraktion, geschämt, dargeboten, aktuel..."
4,Liebe Kolleginnen und Kollegen! Es ist vorhin ...,FDP,"[liebe, vorhin, thema, einlagensicherung, bank...","[vorhin, thema, einlagensicherung, bankenunion..."
...,...,...,...,...
36112,Sehr geehrter Herr Präsident! Liebe Kolleginne...,SPD,"[liebe, gerne, versachlichung, debatte, wolf, ...","[gerne, versachlichung, debatte, wolf, europa,..."
36113,Frau Präsidentin! Meine Damen und Herren! Sie ...,GRÜNE,"[hörten, rede, merz, lahmen, thema, thema, vor...","[hörten, rede, merz, lahmen, thema, thema, vor..."
36114,Sehr geehrter Herr Präsident! Meine Damen und ...,AfD,"[jüngsten, gewaltausbrüche, sudan, folgen, kri...","[jüngsten, gewaltausbrüche, sudan, folgen, kri..."
36115,Sehr geehrte Präsidentin! Liebe Kolleginnen un...,LINKE,"[liebe, demokratischen, ehrlich, woche, mensch...","[demokratischen, ehrlich, woche, menschenfeind..."


In [32]:
# re-join tokeized speeches to strings
preprocessed_data["speech_text"] = preprocessed_data["tokenized_woliebe"].apply(lambda tokens: " ".join(tokens))

# drop helper column
preprocessed_data_B = preprocessed_data.drop(columns=["tokenized_speeches","tokenized_woliebe"], axis = 1)

In [34]:
preprocessed_data_B.to_csv("../data/classifier_data_B.csv",index=False)